In [39]:
from typing import Iterable

from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.blob_loaders.schema import Blob, BlobLoader


class AudioBlobLoader(BlobLoader):

    def __init__(self, file_path: str):
        self._file_path = file_path

    def yield_blobs(self) -> Iterable[Blob]:
        loader = FileSystemBlobLoader(self._file_path)
        for blob in loader.yield_blobs():
            yield blob


In [40]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import OpenAIWhisperParser
import time



start_time = time.time()

file_path = "../data/stt_24_42min.mp3"

# 동영상을 텍스트로 변환
loader = GenericLoader(AudioBlobLoader(file_path), OpenAIWhisperParser(api_key=OPENAI_API_KEY, response_format="srt"))

docs = loader.load()

end_time = time.time()
print(f"수행 시간: {end_time - start_time:.4f}초")

Transcribing part 1!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


Transcribing part 2!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"


수행 시간: 83.5735초


In [41]:
print(type(docs))
print(docs)

<class 'list'>
[Document(metadata={'source': '../data/stt_24_42min.mp3', 'chunk': 0}, page_content='1\n00:00:00,000 --> 00:00:11,240\n자 여러분들 반갑습니다 골드배터리 입니다\n\n2\n00:00:11,240 --> 00:00:15,600\n자 이번엔 뱀스나이크 언제든 서바이벌 개발의 첫번째 시간으로써\n\n3\n00:00:15,600 --> 00:00:17,320\n유니티를 한번 설치해 보고\n\n4\n00:00:17,320 --> 00:00:22,840\n프로젝트 시합과 그 다음에 플레이어를 간단하게 만들어 보는 방법에 대해서\n\n5\n00:00:22,840 --> 00:00:25,120\n같이 한번 진행 좀 해보도록 하겠습니다\n\n6\n00:00:25,120 --> 00:00:28,040\n그래서 여러분들 네이버나 아니면 구글에다가\n\n7\n00:00:28,040 --> 00:00:32,240\n유니티 검색하셔서 이렇게 공식 홈페이지에 들어오시면 되는데요\n\n8\n00:00:32,240 --> 00:00:35,080\n유니티.com.kr 입니다\n\n9\n00:00:35,080 --> 00:00:37,799\n보시면 여기 유니티 다운로드 버튼이 있어요\n\n10\n00:00:37,799 --> 00:00:40,180\n자 들어가시구요\n\n11\n00:00:40,180 --> 00:00:43,279\n그러면 여기 또 윈도우용 다운로드가 있습니다\n\n12\n00:00:43,279 --> 00:00:46,959\n아 물론 또 이제 iOS용 다운로드도 아래에서 할 수 있으니까요\n\n13\n00:00:46,959 --> 00:00:49,240\n확인을 하시면 좋을 것 같습니다\n\n14\n00:00:49,240 --> 00:00:54,160\n그래서 버튼 눌러서 다운로드 하시구요\n\n15\n00:00:54,160 --> 00:00:57,799\n

In [42]:
print(len(docs))

2


In [46]:
import re
from typing import List
from domain import YouTubeScript, YouTubeScriptChunk
from langchain.schema import Document

def timestamp_to_seconds(timestamp: str) -> float:
    """타임스탬프 (00:00:00,000) 형식을 초(float)로 변환"""
    h, m, s_ms = timestamp.split(":")
    s, ms = s_ms.split(",")
    return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000.0

def clean_text(text: str) -> str:
    """텍스트에서 불필요한 숫자 및 라인 번호 제거"""
    text = re.sub(r"^\d+\s*", "", text)  # 앞쪽 라인 번호 제거
    #text = re.sub(r"\s*\d+\s*", " ", text)  # 문장 중간 숫자(단독) 제거
    return text.strip()

def extract_chunks(document: Document) -> List[YouTubeScriptChunk]:
    """Document 객체에서 YouTubeScriptChunk 리스트를 추출하는 함수"""

    chunks = []
    lines = document.page_content.split("\n")

    timestamp_pattern = re.compile(r"(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})")

    i = 0
    while i < len(lines):
        match = timestamp_pattern.search(lines[i])
        if match:
            start_time = timestamp_to_seconds(match.group(1))
            end_time = timestamp_to_seconds(match.group(2))

            text = []
            i += 1
            while i < len(lines) and not timestamp_pattern.search(lines[i]):
                cleaned_line = clean_text(lines[i].strip())  # 불필요한 숫자 제거
                if cleaned_line:  # 빈 문자열 제거
                    text.append(cleaned_line)
                i += 1

            if text:
                chunk = YouTubeScriptChunk(timestamp=(start_time, end_time), text=" ".join(text))
                chunks.append(chunk)
        else:
            i += 1

    return chunks

def convert_documents_to_youtube_script(documents: List[Document]) -> YouTubeScript:
    """Document 객체 리스트를 YouTubeScript 객체로 변환하는 함수"""

    all_chunks = []
    full_script = ""

    for document in documents:
        chunks = extract_chunks(document)
        all_chunks.extend(chunks)
        full_script += " ".join(chunk.text for chunk in chunks) + " "

    return YouTubeScript(script=full_script.strip(), chunks=all_chunks)

In [47]:
script = convert_documents_to_youtube_script(docs)
print(script.to_dict())

{'text': '자 여러분들 반갑습니다 골드배터리 입니다 자 이번엔 뱀스나이크 언제든 서바이벌 개발의 첫번째 시간으로써 유니티를 한번 설치해 보고 프로젝트 시합과 그 다음에 플레이어를 간단하게 만들어 보는 방법에 대해서 같이 한번 진행 좀 해보도록 하겠습니다 그래서 여러분들 네이버나 아니면 구글에다가 유니티 검색하셔서 이렇게 공식 홈페이지에 들어오시면 되는데요 유니티.com.kr 입니다 보시면 여기 유니티 다운로드 버튼이 있어요 자 들어가시구요 그러면 여기 또 윈도우용 다운로드가 있습니다 아 물론 또 이제 iOS용 다운로드도 아래에서 할 수 있으니까요 확인을 하시면 좋을 것 같습니다 그래서 버튼 눌러서 다운로드 하시구요 그러면 유니티 에디터 설치 파일이 나오는 게 아니고 유니티 허브 설치 파일이 나옵니다 자 요거 누르셔서 유니티 허브부터 설치를 하시는 게 좋겠습니다 그래서 설치 동의함 눌러주시구요 자 한가지 제가 추천드리는 사항은요 이 유니티 허브나 아니면 다른거 설치하실 때 보통은 주 드라이버 내에다가 바로 여러분들이 따로 이제 뭐 유니티 허브 프로젝트 폴드를 만드시고 거기에 설치하시는 것을 추천드립니다 그래야 이 유니티 파일들을 관리하기가 상당히 쉽거든요 그래서 이런식으로 깔끔하게 정리해서 설치를 하시는 것을 추천드립니다 그래서 설치를 완료 했으면요 자 요렇게 유니티 허브가 나올 겁니다 자 지금 버전은 유니티 허브 3.3 버전이네요 가장 첫번째 해야 하는 것은 바로 로그인 입니다 여러분들은 당연히 유니티 쓰시면은 유니티 개인 계정 로그인 계정이 필요하구요 그거를 여기에서 로그인 해주셔야 됩니다 그렇게 되면은 여러분들 라이센스를 다 가지게 되는데요 개인 라이센스 있는거 확인하시고 자 지금 여기 유니티 허브잖아요 우리가 보통 말하고 있는 유니티는 유니티 에디터 입니다 그래서 에디터는 여기 설치 탭에 보시면은 이렇게 설치를 할 수 있어요 저는 이미 2022.2 버전 그리고 2021 버전 같이 설치가 되어 있습니다 자 그런데 이 에디터도 허브와 마찬가지로 여러분